In [2]:
import os

In [8]:
%pwd

'd:\\Projects\\Text_Summarizer\\research'

In [9]:
os.chdir('../')

In [10]:
os.listdir()

['.git',
 '.github',
 '.gitignore',
 'app.py',
 'artifacts',
 'config',
 'Dockerfile',
 'LICENSE',
 'logs',
 'main.py',
 'params.yaml',
 'README.md',
 'requirements.txt',
 'research',
 'setup.py',
 'src',
 'template.py',
 'Text_Summarization.ipynb',
 'venv']

In [11]:
%pwd

'd:\\Projects\\Text_Summarizer'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int



In [13]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories
from src.textSummarizer.entity import DataTransformationConfig

In [14]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
          config = self.config.model_trainer
          params = self.params.TrainingArguments

          create_directories([config.root_dir])

          model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

          return model_trainer_config

In [15]:
#components

from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk
import torch



d:\Projects\Text_Summarizer\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-03-31 20:35:19,525:  INFO: PyTorch version 2.6.0 available.]


In [16]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir = self.config.root_dir,
            num_train_epochs= 1,
            warmup_steps= 500,
            per_device_train_batch_size= 1,
            per_device_eval_batch_size= 1,
            weight_decay= 0.01,
            logging_steps= 10,
            evaluation_strategy='steps',
            eval_steps= 500,
            save_steps= 1e6,
            gradient_accumulation_steps= 16
        )

        trainer = Trainer(model = model_pegasus, args = trainer_args,
                          tokenizer = tokenizer,
                          data_collator = seq2seq_data_collator,
                          train_dataset = dataset_samsum_pt["test"],
                          eval_dataset = dataset_samsum_pt["validation"])
        
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [17]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()

except Exception as e:
    raise e

[2025-03-31 20:36:01,137:  INFO: yaml file loaded successfully: config\config.yaml]
[2025-03-31 20:36:01,164:  INFO: yaml file loaded successfully: params.yaml]
[2025-03-31 20:36:01,170:  INFO: Directory created at artifacts]
[2025-03-31 20:36:01,173:  INFO: Directory created at artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\Projects\Text_Summarizer\venv\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ASUS\AppData\Local\Temp\ipykernel_6076\979518274.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model = model_pegasus, args = trainer_args,


Step,Training Loss,Validation Loss


d:\Projects\Text_Summarizer\venv\Lib\site-packages\transformers\modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [1]:
pip show accelerate

Name: accelerate
Version: 1.5.2
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: d:\Projects\Text_Summarizer\venv\Lib\site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show transformers

Name: transformers
Version: 4.50.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: d:\Projects\Text_Summarizer\venv\Lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade accelerate==0.26.0

  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.5.2
    Uninstalling accelerate-1.5.2:
      Successfully uninstalled accelerate-1.5.2
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade transformers[torch]

Note: you may need to restart the kernel to use updated packages.
